# Processing images with oBIF

In [217]:
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC


## Filters

Firstly, we need to implement some filters that will be used by the mtBif class for generating the directions of the image. Mainly, we will use the Gaussian Derivative of 0th, 1st and 2nd order

In [218]:
def mtDownsampleFilter(inFilter, scaleFactor):
    """
    mtDownsampleFilter
    Downsamples a 1D filter by averaging across scaleFactor bins.

    Parameters:
    - inFilter (1D array): Input filter to be downsampled
    - scaleFactor (int): Factor by which to downsample the input filter. Defines the
                        number of bins to average over and collapse
                        into a single bin in the output filter

    Returns:
    - outFilter (1D array): Downsampled filter

    Usage: outFilter = mtDownsampleFilter(inFilter, scaleFactor)
    """
    numBins = len(inFilter)
    outSize = numBins // scaleFactor
    outFilter = np.zeros(outSize)

    for i in range(outSize):
        outFilter[i] = np.mean(inFilter[i * scaleFactor : (i + 1) * scaleFactor])

    return outFilter


In [219]:
def mtGaussianDerivativeFilter1d_s0(x, sigma):
    """
    mtGaussianDerivativeFilter1d_s0
    Generate discrete zeroth-order 1D Gaussian Derivative filter

    Parameters:
    - x (1D array): array of x values
    - sigma (float): Standard deviation of Gaussian used to generate filter

    Returns:
    - s0 (1D array): zeroth order 1D filter

    Usage: s0 = mtGaussianDerivativeFilter1d_s0(x, sigma)
    """
    Cs = 1 / (np.sqrt(2 * np.pi) * sigma)
    s0 = Cs * np.exp(-(x**2) / (2 * sigma**2))
    return s0

def mtGaussianDerivativeFilter1d_s1(x, sigma):
    """
    mtGaussianDerivativeFilter1d_s1
    Generate discrete first-order 1D Gaussian Derivative filter

    Parameters:
    - x (1D array): array of x values
    - sigma (float): Standard deviation of Gaussian used to generate filter

    Returns:
    - s1 (1D array): first order 1D filter

    Usage: s1 = mtGaussianDerivativeFilter1d_s1(x, sigma)
    """
    s0 = mtGaussianDerivativeFilter1d_s0(x, sigma)
    s1 = (x / (sigma**2)) * s0
    return s1

def mtGaussianDerivativeFilter1d_s2(x, sigma):
    """
    mtGaussianDerivativeFilter1d_s2
    Generate discrete second-order 1D Gaussian Derivative filter

    Parameters:
    - x (1D array): array of x values
    - sigma (float): Standard deviation of Gaussian used to generate filter

    Returns:
    - s2 (1D array): second order 1D filter

    Usage: s2 = mtGaussianDerivativeFilter1d_s2(x, sigma)
    """
    s0 = mtGaussianDerivativeFilter1d_s0(x, sigma)
    s2 = ((x**2 - sigma**2) / (sigma**4)) * s0
    return s2

def mtGaussianDerivativeFilters1d(sigma):
    """
    mtGaussianDerivativeFilters1d
    Generates discrete zeroth, first, and second order 1D Gaussian Derivative filters

    Parameters:
    - sigma (float): Standard deviation of Gaussian used to generate filters

    Internal parameters:
    - scaleFactor: Number of bins in higher resolution filter for each bin in
                   original resolution filter
    - widthMulti: Filters are centered on the middle bin. The extent of the filter
                  either side of this central pixel is widthMulti standard deviations

    Returns:
    - s0 (1D array): zeroth order 1D filter
    - s1 (1D array): first-order 1D filter
    - s2 (1D array): second-order 1D filter

    Usage: s0, s1, s2 = mtGaussianDerivativeFilters1d(sigma)
    """

    # Force numeric parameters to be float to avoid errors
    sigma = float(sigma)

    scaleFactor = 10
    widthMulti = 5
    halfWidth = (np.ceil(sigma) * widthMulti) + 1

    # Set x values for each bin of high resolution filter. Note that the x limits
    # are the same, we just generate more bins between them
    xHighRes = np.arange(-halfWidth, halfWidth + 1, 1 / scaleFactor)

    # Zeroth order filter
    s0HighRes = mtGaussianDerivativeFilter1d_s0(xHighRes, sigma)
    s0 = mtDownsampleFilter(s0HighRes, scaleFactor)
    # First-order filter
    s1R = mtGaussianDerivativeFilter1d_s1(xHighRes, sigma)
    s1 = mtDownsampleFilter(s1R, scaleFactor)
    # Second-order filter
    s2R = mtGaussianDerivativeFilter1d_s2(xHighRes, sigma)
    s2 = mtDownsampleFilter(s2R, scaleFactor)

    return s0, s1, s2


In [220]:
def mtSeparableFilter2(xFilter, yFilter, input, mode="mirror"):
    """
    mtSeparableFilter2
    Performs 2D filtering of a 2D matrix input by a separable 2D filter by
    filtering by the x and y filter components in turn and combining the results

    Parameters:
    - xFilter (1D array): 1D filter comprising the x-component of a separable 2D filter
    - yFilter (1D array): 1D filter comprising the y-component of a separable 2D filter
    - input (2D array): The 2D input matrix to be filtered
    - mode (str): The filter mode. Options are:
        - 'cyclic': Returns an output the same size and the input. The input is
                    extended in x and y prior to filtering, with the extended boundaries
                    filled with pixels "wrapped round" from the opposite edge of the input
        - 'mirror': Returns an output the same size and the input. The input is
                    extended in x and y, prior to filtering, with the extended boundaries
                    filled with pixels "reflected" from the edge of the input.
        - Any other valid "mode" argument to SciPy's convolve2d function. These
                    options will all extend the image in x and y prior to filtering, with the
                    extended boundaries filled according to the mode.
                    For more information, see the documentation of SciPy's convolve2d function.

    Returns:
    - output (2D array): Filtered output (same size as input)
    """
    
    # Check x and y filters are 1D vectors
    if xFilter.ndim != 1 or yFilter.ndim != 1:
        raise ValueError("x and y filters must be 1D vectors")
    # Check x and y filters have an odd length
    if len(xFilter) % 2 == 0 or len(yFilter) % 2 == 0:
        raise ValueError("x and y filters must be of odd length")

    # Ensure x is a row vector and y is a column vector
    xFilter = np.reshape(xFilter, (1, len(xFilter)))
    yFilter = np.reshape(yFilter, (len(yFilter), 1))

    if mode.lower() == "cyclic":
        # If cyclic mode selected, extend image in x (for x filter pass) and y (for y filter pass)
        # and fill extended padding with pixels from opposite side of image.
        # Using convolve2d in 'valid' mode on the padded image returns an output with the same dimensions as the unpadded input image.
        xOut = convolve2d(
            input, np.flip(xFilter, axis=1), mode="valid", boundary="wrap"
        )
        output = convolve2d(
            xOut, np.flip(yFilter, axis=0), mode="valid", boundary="wrap"
        )
    elif mode.lower() == "mirror":
        # If mirror mode selected, extend image in x (for x filter pass) and y (for y filter pass)
        # and fill extended padding with reflected boundary pixels.
        # Using convolve2d in 'valid' mode on the padded image returns an output with the same dimensions as the unpadded input image.
        xOut = convolve2d(
            input, np.flip(xFilter, axis=1), mode="valid", boundary="symm"
        )
        output = convolve2d(
            xOut, np.flip(yFilter, axis=0), mode="valid", boundary="symm"
        )
    else:
        # Otherwise just apply convolve2d for x and y filters, passing selected mode
        # Modes supported by convolve2d are:
        # 'same' - (default) returns the central part of the correlation that is the same size as the image.
        # 'valid' - returns only those parts of the correlation that are computed without the zero-padded edges, size(output) < size(image).
        # 'full' - returns the full 2-D correlation, size(output) > size(image).
        output = convolve2d(
            convolve2d(input, xFilter, mode=mode, boundary="fill"),
            yFilter,
            mode=mode,
            boundary="fill",
        )

    return output


In [221]:
def mtFilter2d(xyFilter, input, mode, method=None):
    """
    Performs 2D filtering of a 2D matrix input by a 2D filter

    Parameters:
    - xyFilter: 2D filter. Filter must be of odd size in both dimensions (and therefore have a centre pixel)
    - input: The 2D input matrix to be filtered
    - mode: The filter mode. Options are:
        - 'cyclic': Returns an output the same size and the input. The input is extended in x and y prior to filtering, 
                    with the extended boundaries filled with pixels "wrapped round" from the opposite edge of the input
        - 'mirror': Returns an output the same size as the input. The input is extended in x and y, prior to filtering, 
                    with the extended boundaries filled with pixels "reflected" from the edge of the input.
        - 'valid': Performs a 'valid' filter operation (i.e. no filtered values for border pixels, but returns an 
                    output the same size as the input by padding the valid output with NaN
    - method: Method to use to perform filtering operation. Optional. If not provided, defaults to 'conv2'
        - 'conv2': use built-in SciPy convolve2d method
        - 'convolve2': Not available in SciPy. Would require a third-party library or implementation
        - 'convnfft': Not available in SciPy. Would require a third-party library or implementation

    Returns:
    - output: Filtered output. Same size as input.
    """

    if method is None:
        method = 'conv2'


    height, width = xyFilter.shape
    # check filter is odd in both dimensions (therefore has a centre pixel)
    if not np.all(np.mod([width, height], 2) == 1):
        raise ValueError('filter must have dimensions of odd length')

    xPad = (width - 1) // 2
    yPad = (height - 1) // 2
    rawMode = 'valid'
    postPadNans = False  # Used for 'padded' mode

    # If custom mode, amend image and mode then apply filter2/conv2
    if mode.lower() == 'cyclic':
        # If cyclic mode selected, extend image in x and y and fill extended padding
        # with pixels from opposite side of image. Using filter2 in 'valid' mode on the
        # padded image returns an output with the same dimensions as the unpadded input
        # image.
        input = np.pad(input, ((yPad, yPad), (xPad, xPad)), mode='wrap')
    elif mode.lower() == 'mirror':
        # If mirror mode selected, extend image in x and y and fill extended padding
        # with reflected boundary pixels. Using filter2 in 'valid' mode on the padded
        # image returns an output with the same dimensions as the unpadded input image.
        input = np.pad(input, ((yPad, yPad), (xPad, xPad)), mode='reflect')
    elif mode.lower() == 'padded':
        postPadNans = True

    # Perform correlation by flipping filter and doing convolution
    if method.lower() == 'conv2':
        output = convolve2d(input, np.flip(xyFilter), mode=rawMode)
    else:
        raise ValueError('Invalid method argument.')

    if postPadNans:
        output = np.pad(output, ((yPad, yPad), (xPad, xPad)), mode='constant', constant_values=np.nan)

    return output


## oBIF class

We will create a class for oBIF objects. This means that we are going to process the image provided and transform it into an image of directions, using the previously defined filters. Also, when displaying this object, we have assigned each direction with a color and it will be displayed as a colour map.

In [222]:
class mtBifs:
    def __init__(self, inputImage, blurWidth, flatnessThreshold):
        # Convert color image to grayscale if needed
        if len(inputImage.shape) == 3:
            inputImage = cv2.cvtColor(inputImage, cv2.COLOR_BGR2GRAY)

        # Rename input parameters to terms used in BIF papers
        # Force numeric parameters to be floats to avoid mixing integers and floats
        self.sigma = float(blurWidth)
        self.gamma = float(flatnessThreshold)

        # Generate filter responses
        self.L, Lx, Ly, Lxx, Lyy, Lxy = self.dtgFilterResponsesFromImage(
            inputImage, self.sigma
        )

        # Generate and set BIF classes
        self.Class = self.bifClassesFromFilterResponses(
            self.sigma, self.gamma, self.L, Lx, Ly, Lxx, Lyy, Lxy
        )

        # Generate and set BIF orientations
        self.Vx, self.Vy = self.bifOrientationsFromFilterResponses(
            self.L, Lx, Ly, Lxx, Lyy, Lxy
        )

    @staticmethod
    def dtgFilterResponsesFromImage(inputImage, sigma):
        # Generate the 1D Gaussian Derivative filters used to calculate BIFs
        # s0 = zeroth order 1D filter
        # s1 = first-order 1D filter
        # s2 = second-order 1D filter
        s0, s1, s2 = mtGaussianDerivativeFilters1d(sigma)

        # Calculate 2D filter responses over the image using the 1D filters
        # Pad extended boundary so filter response is same size as input
        # image and pad boundary with reflected edge pixels
        filterMode = "mirror"
        # zeroth order filter
        L = mtSeparableFilter2(s0, s0, inputImage, filterMode)
        # first-order in x, zeroth in y
        Lx = mtSeparableFilter2(s1, s0, inputImage, filterMode)
        # first-order in y, zeroth in x
        Ly = mtSeparableFilter2(s0, s1, inputImage, filterMode)
        # second-order in x, zeroth in y
        Lxx = mtSeparableFilter2(s2, s0, inputImage, filterMode)
        # second-order in y, zeroth in x
        Lyy = mtSeparableFilter2(s0, s2, inputImage, filterMode)
        # first-order in x and y
        Lxy = mtSeparableFilter2(s1, s1, inputImage, filterMode)

        return L, Lx, Ly, Lxx, Lyy, Lxy

    @staticmethod
    def bifClassesFromFilterResponses(sigma, gamma, L, Lx, Ly, Lxx, Lyy, Lxy):
        # Compute BIF classes
        numBifClasses = 7
        numYs, numXs = L.shape
        jetScore = np.zeros((numYs, numXs, numBifClasses))

        # 1: flat (pink)
        jetScore[:, :, 0] = gamma * L
        # 2: gradient (grey)
        jetScore[:, :, 1] = sigma * np.sqrt(Lx**2 + Ly**2)

        # Second order BIFs are calculated from Hessian eigenvalues.
        # The formulation below has been chosen to be numerically stable
        # as some issues were encountered due to numerical precision
        # issues when using some other formulations.
        eigVal1 = (Lxx + Lyy + np.sqrt((Lxx - Lyy) ** 2 + 4 * Lxy**2)) / 2
        eigVal2 = (Lxx + Lyy - np.sqrt((Lxx - Lyy) ** 2 + 4 * Lxy**2)) / 2

        # 3: dark blob (black)
        jetScore[:, :, 2] = sigma**2 * (eigVal1 + eigVal2) / 2
        # 4: light blob (white)
        jetScore[:, :, 3] = -(sigma**2) * (eigVal1 + eigVal2) / 2
        # 5: dark line (blue)
        jetScore[:, :, 4] = sigma**2 * eigVal1 / np.sqrt(2)
        # 6: light line (yellow)
        jetScore[:, :, 5] = -(sigma**2) * eigVal2 / np.sqrt(2)
        # 7: saddle (green)
        jetScore[:, :, 6] = sigma**2 * (eigVal1 - eigVal2) / 2

        # Get maximum BIF score at each pixel (index in third dimension
        # corresponds to integer code for BIF class)
        bifClasses = np.argmax(jetScore, axis=2) + 1

        return bifClasses

    def bifOrientationsFromFilterResponses(self, L, Lx, Ly, Lxx, Lyy, Lxy):
        # Compute unit vector orientations for first order BIF classes
        vx1 = Lx
        vy1 = Ly
        norm1 = np.sqrt(vx1**2 + vy1**2)
        vx1 = np.divide(vx1, norm1, out=np.zeros_like(vx1), where=norm1 != 0)
        vy1 = np.divide(vy1, norm1, out=np.zeros_like(vy1), where=norm1 != 0)

        # Compute unit vector orientations for second order BIF classes
        vx2 = -(-Lxx + Lyy + np.sqrt((Lxx - Lyy) ** 2 + 4 * Lxy**2)) / (2 * Lxy)
        vy2 = np.ones_like(vx2)
        norm2 = np.sqrt(vx2**2 + vy2**2)
        vx2 = np.divide(vx2, norm2, out=np.zeros_like(vx2), where=norm2 != 0)
        vy2 = np.divide(vy2, norm2, out=np.zeros_like(vy2), where=norm2 != 0)

        # Handle cases where Lxy is zero
        vertMask = Lxx > Lyy
        horzMask = Lyy > Lxx
        zero_mask = Lxy == 0
        vx2[zero_mask & vertMask] = 0
        vy2[zero_mask & vertMask] = 1
        vx2[zero_mask & horzMask] = 1
        vy2[zero_mask & horzMask] = 0

        # Handle light ridges
        lightMask2 = self.Class == 6
        vx2Old = vx2.copy()
        vy2Old = vy2.copy()
        vx2[lightMask2] = vy2Old[lightMask2]
        vy2[lightMask2] = -vx2Old[lightMask2]

        # Assign orientations to output variables
        vx = np.where(self.Class == 2, vx1, vx2)
        vy = np.where(self.Class == 2, vy1, vy2)

        # Another approach to assigning orientations
        # vx = np.zeros_like(vx1)
        # vy = np.zeros_like(vy1)

        # mask1 = self.Class == 2
        # vx[mask1] = vx1[mask1]
        # vy[mask1] = vy1[mask1]

        # mask2 = (self.Class >= 5) & (self.Class <= 7)
        # vx[mask2] = vx2[mask2]
        # vy[mask2] = vy2[mask2]

        return vx, vy

    @staticmethod
    def colourMap():
        # Generates colour map for use when displaying BIFs
        #
        # OUTPUTS:
        # colourMap: Follows the format of built-in Matlab colour maps. Row N+1 defines
        #            the RGB colour to display for matrix elements with value N. Colours
        #            as per the BIF journal papers from Crosier and Griffin with the
        #            addition of mapping for value 0 in row 1. This is not a valid BIF
        #            class but is required for a valid colour map.
        #               0 = invalid (cyan)
        #               1 = flat (pink)
        #               2 = gradient (grey)
        #               3 = dark blob (black)
        #               4 = light blob (white)
        #               5 = dark line (blue)
        #               6 = light line (yellow)
        #               7 = saddle (green)
        #
        # USAGE: colourMap = mtBifs.colourMap()

        # Define color map for displaying BIFs
        bif_cyan = [0, 0.5, 0.5]
        bif_pink = [1, 0.7, 0.7]
        bif_grey = [0.6, 0.6, 0.6]
        bif_black = [0, 0, 0]
        bif_white = [1, 1, 1]
        bif_blue = [0.1, 0.1, 1]
        bif_yellow = [0.9, 0.9, 0]
        bif_green = [0, 1, 0]

        return np.array(
            [
                bif_cyan,
                bif_pink,
                bif_grey,
                bif_black,
                bif_white,
                bif_blue,
                bif_yellow,
                bif_green,
            ]
        )

    @staticmethod
    def drawBifDir2d(self, x, y, xScale, yScale, lineWidth, bifClass, vx, vy):

        # Draw BIF orientation marks
        if bifClass in [0, 1, 3, 4]:
            return  # Do nothing. No direction associated with these BIF classes

        X1 = [x - (xScale * 0.5) * vx, x + (xScale * 0.5) * vx]
        Y1 = [y - (yScale * 0.5) * vy, y + (yScale * 0.5) * vy]

        if bifClass == 2:
            X1 = [x, x + (xScale * 0.5) * vx]
            Y1 = [y, y + (yScale * 0.5) * vy]
            X2 = [x - (xScale * 0.5) * vx, x]
            Y2 = [y - (yScale * 0.5) * vy, y]
            plt.plot(X1, Y1, color="w", linewidth=lineWidth)
            plt.plot(X2, Y2, color="k", linewidth=lineWidth)
        elif bifClass == 5:
            plt.plot(X1, Y1, color="k", linewidth=lineWidth)
        elif bifClass == 6:
            plt.plot(X1, Y1, color="w", linewidth=lineWidth)
        elif bifClass == 7:
            X2 = [x - (xScale * 0.5) * vy, x + (xScale * 0.5) * vy]
            Y2 = [y + (yScale * 0.5) * vx, y - (yScale * 0.5) * vx]
            plt.plot(X1, Y1, color="k", linewidth=lineWidth)
            plt.plot(X2, Y2, color="w", linewidth=lineWidth)

    def getSnippet(self, rows, cols):
        """
        Creates a new mtBifs object with BIF data for specified rows and columns

        Args:
            rows (slice or list or numpy.ndarray): Rows for which to get BIF data
            cols (slice or list or numpy.ndarray): Columns for which to get BIF data

        Returns:
            mtBifs: mtBifs object containing BIF data for specified rows and columns
        """
        bifSnippet = mtBifs.__new__(mtBifs)  # Create a new mtBifs object
        bifSnippet.sigma = self.sigma  # Copy sigma value
        bifSnippet.gamma = self.gamma  # Copy gamma value

        # Convert rows and cols to numpy arrays if they are not already
        rows = np.asarray(rows)
        cols = np.asarray(cols)

        # Slice and copy relevant portions of Class, Vx, and Vy arrays
        bifSnippet.Class = self.Class[rows, cols]
        bifSnippet.Vx = self.Vx[rows, cols]
        bifSnippet.Vy = self.Vy[rows, cols]

        return bifSnippet

    def show(self, showOrientation=False):
        # Set BIF class colour map
        bifMap = self.colourMap()

        # Set all elements with invalid BIF classes to 0
        minValidBifClass = 1
        maxValidBifClass = 7
        bifClasses = self.Class.copy()
        bifClasses[
            (bifClasses < minValidBifClass) | (bifClasses > maxValidBifClass)
        ] = 0

        # Show BIF classes with colour map
        bifImage = bifMap[bifClasses]

        # Add direction marks if requested
        if showOrientation:
            numRows, numCols = self.Class.shape
            scale = 0.6
            lineWidth = 2
            for r in range(numRows):
                for c in range(numCols):
                    self.drawBifDir2d(
                        self,
                        c,
                        r,
                        scale,
                        scale,
                        lineWidth,
                        self.Class[r, c],
                        self.Vx[r, c],
                        self.Vy[r, c],
                    )

        return bifImage

    def generate_histogram(self):
        """
        Generates the oBIF histogram for the current mtBifs object.

        Returns:
            numpy.ndarray: The oBIF histogram.
        """
        # Initialize histogram bins
        num_bins = 23  # From the given formula 5n + 3
        histogram = np.zeros(num_bins)

        # Iterate over each pixel in the image
        for row in range(self.Class.shape[0]):
            for col in range(self.Class.shape[1]):
                # Determine the orientation bin index based on oBIF orientation and class
                if self.Class[row, col] == 2:  # Gray class
                    orientation = (
                        np.arctan2(self.Vy[row, col], self.Vx[row, col]) / np.pi
                    )
                    if 0.125 >= orientation > -0.125:
                        bin_index = 1  # 1st quadrant
                    elif 0.375 >= orientation > 0.125:
                        bin_index = 2  # 2nd quadrant
                    elif 0.625 >= orientation > 0.375:
                        bin_index = 3  # 3rd quadrant
                    elif 0.875 >= orientation > 0.625:
                        bin_index = 4  # 4th quadrant
                    elif -0.125 >= orientation > -0.375:
                        bin_index = 8  # 8th quadrant
                    elif -0.375 >= orientation > -0.625:
                        bin_index = 7  # 7th quadrant
                    elif -0.625 >= orientation > -0.875:
                        bin_index = 6  # 6th quadrant
                    else:
                        bin_index = 5  # 5th quadrant
                elif self.Class[row, col] == 3:  # black class
                    bin_index = 9
                elif self.Class[row, col] == 4:  # white class
                    bin_index = 10
                elif self.Class[row, col] == 5:  # Blue class
                    orientation = (
                        np.arctan2(self.Vy[row, col], self.Vx[row, col]) / np.pi
                    )
                    if 0.875 >= orientation > 0.625 or -0.125 >= orientation > -0.375:
                        bin_index = 14  # 4th and 8th quadrant
                    elif 0.625 >= orientation > 0.375 or -0.375 >= orientation > -0.625:
                        bin_index = 13  # 3th and 7th quadrant
                    elif 0.375 >= orientation > 0.125 or -0.625 >= orientation > -0.875:
                        bin_index = 12  # 2nd and 6th quadrant
                    else:
                        bin_index = 11  # 1st and 5th quadrant
                elif self.Class[row, col] == 6:  # Yellow class
                    orientation = (
                        np.arctan2(self.Vy[row, col], self.Vx[row, col]) / np.pi
                    )
                    if 0.875 >= orientation > 0.625 or -0.125 >= orientation > -0.375:
                        bin_index = 18  # 4th and 8th quadrant
                    elif 0.625 >= orientation > 0.375 or -0.375 >= orientation > -0.625:
                        bin_index = 17  # 3th and 7th quadrant
                    elif 0.375 >= orientation > 0.125 or -0.625 >= orientation > -0.875:
                        bin_index = 16  # 2nd and 6th quadrant
                    else:
                        bin_index = 15  # 1st and 5th quadrant
                elif self.Class[row, col] == 7:  # Green class
                    orientation = (
                        np.arctan2(self.Vy[row, col], self.Vx[row, col]) / np.pi
                    )
                    if 0.875 >= orientation > 0.625 or -0.125 >= orientation > -0.375:
                        bin_index = 22  # 4th and 8th quadrant
                    elif 0.625 >= orientation > 0.375 or -0.375 >= orientation > -0.625:
                        bin_index = 21  # 3th and 7th quadrant
                    elif 0.375 >= orientation > 0.125 or -0.625 >= orientation > -0.875:
                        bin_index = 20  # 2nd and 6th quadrant
                    else:
                        bin_index = 19  # 1st and 5th quadrant
                else:  # invalid classes y pink class
                    continue

                # Increment the corresponding bin in the histogram
                histogram[bin_index] += 1

        # Normalize the histogram
        histogram /= np.sum(histogram)

        return histogram


## Plot images
We have plotted the original image with the processed one

In [223]:
def mtShowCaseBifs(title_text, image, bifs, zoom_rows, zoom_cols):
    """
    mtShowCaseBifs
    Displays image and BIFs for visual inspection. Three panels are shown as follows:
    - Left: entire input
    - Middle: BIF classes for entire image
    - Right: BIF classes and orientations for the specified rows and columns

    Parameters:
    - title_text (str): Text to display in figure window
    - image (numpy.ndarray): Input image for which BIFs have been calculated
    - bifs (mtBifs object): Object with BIF classes and orientations for entire image
    - zoom_rows (list): Rows to zoom in and show BIF orientations for
    - zoom_cols (list): Columns to zoom in and show BIF orientations for

    NOTE: It is recommended to show orientations only for small image areas
    (say up to about 50x50 pixels). For larger areas, it can be hard to
    distinguish the orientations and image display features tend to get a bit slow.

    Usage: mtShowCaseBifs(title_text, image, bifs, zoom_rows, zoom_columns)
    """

    # Open figure
    plt.figure(title_text, figsize=(12, 8))

    # Show test image
    plt.subplot(2, 2, 1)
    num_dims = len(image.shape)
    if num_dims == 2:
        # Convert greyscale image to RGB image
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.imshow(image)
    plt.xlabel("Test image")

    # Show BIF classes for whole image
    plt.subplot(2, 2, 2)
    bif_image = bifs.show()
    plt.imshow(bif_image)
    plt.xlabel("BIF class for all pixels")

    # Show test image
    plt.subplot(2, 2, 3)
    plt.imshow(image[zoom_rows[:, np.newaxis], zoom_cols])
    plt.xlabel("Image for zoomed area")
        
    # Show BIF classes and orientation for zoomed pixel(s)
    plt.subplot(2, 2, 4)
    snippet = bifs.getSnippet(zoom_rows[:, np.newaxis], zoom_cols).show(showOrientation=True)
    plt.imshow(snippet)
    plt.xlabel('BIF class and orientation\nfor zoomed area')

    plt.show()


## Try it out

We have implemented a function to demonstrate the oBif images

In [226]:
def generate_histograms(image_names):
    # Define the different values of sigma and epsilon
    sigma_values = [1, 2, 4, 8, 16]
    epsilon_values = [0.1, 0.01, 0.001]

    # Initialize a dictionary to store the histograms
    histograms = {}

    # Iterate over each image
    for image_name in image_names:
        # Load the image
        im = cv2.imread(image_name)
        if im is None:
            print(f"Failed to load image {image_name}")
            continue
        
        width = int(im.shape[1] // 3)
        height = int(im.shape[0] // 3)
        
        image = im[np.arange(0, height)[:, np.newaxis], np.arange(0, width)]

        # Initialize a list to store the histograms for this image
        image_histograms = []

        # Iterate over each combination of sigma and epsilon values
        for sigma in sigma_values:
            for epsilon in epsilon_values:
                # Generate the oBIFs for the current image
                bifs = mtBifs(image, sigma, epsilon)

                # Generate the histogram for the current image
                histogram = bifs.generate_histogram()
                
                # Check if there are any NaN values in the histogram
                if np.isnan(histogram).any():
                    print(f"Found NaN in histogram for image {image_name} with sigma={sigma} and epsilon={epsilon}")
                    break

                # Append the histogram to the list of histograms for this image
                image_histograms.append(histogram)
            else:
                continue
            break
        else:
            # If a NaN was found in any histogram, skip this image
            if any(np.isnan(histogram).any() for histogram in image_histograms):
                print(f"Skipping image {image_name} due to NaN in histogram")
                continue

            # Concatenate the histograms for this image into one
            histograms[image_name] = np.concatenate(image_histograms)

    return histograms

In [ ]:
def process_images(females_path, males_path):
    # Get all female and male images
    female_images = glob.glob(females_path)
    male_images = glob.glob(males_path)

    # Sort the images numerically
    female_images = sorted(female_images, key=extract_number)
    male_images = sorted(male_images, key=extract_number)

    # Combine the female and male images
    image_names = female_images + male_images

    # Calculate histograms
    histograms = generate_histograms(image_names)

    # Only include images in the set if their histograms do not contain any NaN values
    images = [image_name for image_name in image_names if image_name in histograms.keys()]

    # Get the labels for the images
    labels = ['F' if 'F' in image_name else 'M' for image_name in images]

    # Assuming X contains histograms and y contains corresponding labels (male or female)
    X = np.vstack([histograms[image_name] for image_name in images])
    y = labels

    return X, y


# Function to extract number from image name
def extract_number(image_name):
    match = re.search(r'\d+', image_name)
    return int(match.group()) if match else 0


In [ ]:
def train_and_test_model(females_path, males_path):
    # Assuming X contains histograms and y contains corresponding labels (male or female)
    X, y = process_images(females_path, males_path)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the parameter grid for GridSearchCV
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.1, 0.01, 0.001, 0.0001],
    }

    # Create an SVM classifier
    svm_classifier = SVC(kernel='rbf')

    # Perform GridSearchCV for parameter tuning
    grid_search = GridSearchCV(svm_classifier, param_grid, cv=5)
    grid_search.fit(X_train, y_train)

    # Get the best parameters
    best_params = grid_search.best_params_
    print("Best Parameters:", best_params)

    # Train the SVM classifier with the best parameters
    best_svm_classifier = SVC(kernel='rbf', C=best_params['C'], gamma=best_params['gamma'])
    best_svm_classifier.fit(X_train, y_train)

    # Predict the labels for the test set
    y_pred = best_svm_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Test Accuracy:", accuracy)

    return best_svm_classifier

In [ ]:
def test_experimentation(females_path, males_path, best_svm_classifier):
    # Process the test images
    X_test, y_test = process_images(females_path, males_path)

    # Predict the labels for the test set
    y_pred = best_svm_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Test Accuracy:", accuracy)

    return accuracy

In [ ]:
# Train ai
females_path = './images/Females/*.jpg'
males_path = './images/Males/*.jpg'
best_svm_classifier = train_and_test_model(females_path, males_path)

In [229]:
# Test ai
test_females_path = './test_images/Females/*.jpg'
test_males_path = './test_images/Males/*.jpg'
test_accuracy = test_experimentation(test_females_path, test_males_path, best_svm_classifier)

['./images/Females\\F1.jpg', './images/Females\\F10.jpg', './images/Females\\F100.jpg', './images/Females\\F101.jpg', './images/Females\\F102.jpg', './images/Females\\F103.jpg', './images/Females\\F104.jpg', './images/Females\\F105.jpg', './images/Females\\F106.jpg', './images/Females\\F107.jpg', './images/Females\\F108.jpg', './images/Females\\F109.jpg', './images/Females\\F11.jpg', './images/Females\\F110.jpg', './images/Females\\F111.jpg', './images/Females\\F112.jpg', './images/Females\\F113.jpg', './images/Females\\F114.jpg', './images/Females\\F115.jpg', './images/Females\\F116.jpg', './images/Females\\F117.jpg', './images/Females\\F118.jpg', './images/Females\\F119.jpg', './images/Females\\F12.jpg', './images/Females\\F120.jpg', './images/Males\\M1.jpg', './images/Males\\M10.jpg', './images/Males\\M100.jpg', './images/Males\\M101.jpg', './images/Males\\M102.jpg', './images/Males\\M103.jpg', './images/Males\\M104.jpg', './images/Males\\M105.jpg', './images/Males\\M106.jpg', './im

KeyboardInterrupt: 